## Movies

In [4]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts = "http://@localhost:9200")

# Generic queries to display index/mapping

# show documents of an index; increment "size" argument to display more than 100 docs)
result = es.search(index="movies", size=100)

# show mapping of an index
#result = es.indices.get_mapping(index="movies")

# simple query to mach any filed with any value
query_body = {"query": { "match": { "byline": "Lisa Kennedy"}}}
#result = es.search(index="movies", body=query_body)

result.body

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4460, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'movies',
    '_id': 'vigW-YgBq0sA4CElWyqy',
    '_score': 1.0,
    '_source': {'display_title': 'The Stroll',
     'mpaa_rating': '',
     'critics_pick': 1,
     'byline': 'Devika Girish',
     'headline': '‘The Stroll’ Review: Telling Their Own Stories',
     'summary_short': 'In this documentary, transgender sex workers speak for themselves without sanitizing or sensationalizing their experiences.',
     'publication_date': '2023-06-22',
     'opening_date': '2023-06-21',
     'date_updated': '2023-06-22 14:35:03',
     'link': {'type': 'article',
      'url': 'https://www.nytimes.com/2023/06/22/movies/the-stroll-review-documentary.html',
      'suggested_link_text': 'Read the New York Times Review of The Stroll'},
     'multimedia': {'type': 'mediumThreeByTwo210',
      'src': 'https:/